In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By

from tqdm import notebook
import datetime
import time
import pandas as pd

In [4]:
#드라이버 불러온 후 전체화면 크기로 설정
driver = webdriver.Chrome('./chromedriver.exe')
driver.maximize_window()

### CSV update 코드  
- ```file_name```과 ```header(column)```으로 들어갈 값을 작성

In [5]:
def update_csv(f_name, data_list):
    df = pd.DataFrame(data_list, index=[0])
    df.to_csv(f_name, index=False, encoding = 'utf-8-sig',mode='a')


### jumpit site 크롤링 코드  
- Category 1~21까지 반복하며 나오는 공고들을 하나씩 클릭
-> CSV 형태로 저장 **[col: 회사이름,기술스택, 주요업무, ...]**
- 인터넷 성능에 따라 ```implicityly_wait()```의 시간을 조정하면 될 것 같습니다.

In [9]:
def jumpit_crawling(start_category, start_xpath_location):

    Category_name = [0,'서버/백엔드 개발자','프론트엔드 개발자','웹 풀스택 개발자', '안드로이드 개발자','게임 클라이언트 개발자', '게임 서버 개발자', 'DBA','인공지능/머신러닝', 'devops/시스템 엔지니어',
                     '정보보안 담당자','QA 엔지니어', '개발 PM','HW/임베디드',0,'SW/솔루션','IOS 개발자','웹퍼블리셔','크로스플랫폼 앱개발자','빅데이터 엔지니어','VR/AR/3D','기술지원','블록체인']
    max_category = 22
    max_crawling_data = 1000

    URL = 'https://www.jumpit.co.kr/positions'
    for jobCategory in range(start_category,max_category+1):
        
        if jobCategory == 14:
            continue

        driver.get(URL + f'?jobCategory={jobCategory}') 
        driver.implicitly_wait(time_to_wait=5)
        
        xpath_location = start_xpath_location

        csv_name = f'크롤링_점핏_{jobCategory}.csv'
        df = pd.DataFrame(columns = ['회사명', '지원기간', '공고명', '직무내용', '자격요건', '우대조건', '경력조건', '기술 스택', '출처 URL', '복지','회사소개', '주요업무'])

        #직무명
        job = Category_name[jobCategory]

        #페이지의 공고들을 반복하여 클릭해서 들어감
        for i in notebook.tqdm(range(max_crawling_data), desc = f'{max_crawling_data}개 크롤링'):
            xpath_location += 1
            driver.implicitly_wait(time_to_wait=10)

            #회사명, 채용공고명
            try:
                company_name = driver.find_element_by_xpath(f'//*[@id="root"]/main/div/div[1]/section/div[{xpath_location}]/a/div[3]/div').text
                field_recruitment = driver.find_element_by_xpath(f'//*[@id="root"]/main/div/div[1]/section/div[{xpath_location}]/a/div[3]/h2').text
            
            except:
                df = pd.concat([df, pd.DataFrame(new_row, index=[0])], ignore_index=True)
                df.to_csv(csv_name, index=False)
                break

            #공고 클릭
            driver.find_element(By.XPATH, f'//*[@id="root"]/main/div/div[1]/section/div[{xpath_location}]').click()
            driver.implicitly_wait(time_to_wait=10)

            url = driver.current_url
            skill = driver.find_element_by_xpath(f'//*[@id="root"]/main/div/div[2]/div/section[2]/dl[1]/dd/pre').text
            main_task = driver.find_element_by_xpath(f'//*[@id="root"]/main/div/div[2]/div/section[2]/dl[2]/dd/pre').text
            qualification = driver.find_element_by_xpath(f'//*[@id="root"]/main/div/div[2]/div/section[2]/dl[3]/dd/pre').text
            preference = driver.find_element_by_xpath(f'//*[@id="root"]/main/div/div[2]/div/section[2]/dl[4]/dd/pre').text
            benefit = driver.find_element_by_xpath(f'//*[@id="root"]/main/div/div[2]/div/section[2]/dl[5]/dd/pre').text
            career = driver.find_element_by_xpath(f'//*[@id="root"]/main/div/div[2]/div/section[3]/dl[1]').text

            if driver.find_element_by_xpath(f'//*[@id="root"]/main/div/div[2]/div/section[3]/dl[3]/dt').text == '마감일':
                deadline = driver.find_element_by_xpath(f'//*[@id="root"]/main/div/div[2]/div/section[3]/dl[3]').text
            else: 
                deadline = '상시'

            company_name = driver.find_element_by_xpath(f'//*[@id="root"]/main/div/div[2]/div/section[4]/div[2]/pre').text
   
            new_row = {'회사명':company_name,'지원기간':deadline,'공고명':field_recruitment,'직무내용':job,
                       '자격요건':qualification,'우대조건':preference,'경력조건':career,'기술 스택':skill,
                       '출처 URL':url, '복지':benefit, '회사소개': company_name, '주요업무': main_task}

            df = pd.concat([df, pd.DataFrame(new_row, index=[0])], ignore_index=True)

            #30개에 한 번씩 저장
            if len(df) % 30 == 0:
                df.to_csv(csv_name, index=False)
                print('+30 topic -> CSV file update')

            driver.back()
            driver.implicitly_wait(time_to_wait=3)
        
        if len(df) % 30 != 0:
            df.to_csv(csv_name, index=False)
            print(f'remainder {len(df)%30}  topic -> CSV file update')
            print(f'{job} Category 끝')
            driver.implicitly_wait(time_to_wait=3)

### 데이터 수집 및 시간 측정

In [20]:
#수행해야할 category_number와 xpath_location 입력

start = time.time()
jumpit_crawling(22, 0)
end = time.time()

result = datetime.timedelta(seconds=end-start)
print(result)

+30 topic -> CSV file update

remainder 28  topic -> CSV file update
블록체인 Category 끝
0:00:58.449939


In [1]:
import pandas as pd
r = pd.read_csv('../Data_Pretreatment/Data_result/pre_result_morph.csv')

r.head()

,회사명,공고명,직무내용,자격요건,우대조건,경력조건,출처 URL,복지,회사소개,주요업무
0,혜움랩스,"['Python', 'Developer', '백엔드', '주니어']",웹 개발자,"['최소', '2년', '이상', '혹은', '그', '에', '준', '하는', ...","['MSA', '기반', '의', '시스템', '개발', '경험', '이', '있으...",2,https://www.wanted.co.kr/wd/167436,"['#', '몰입', '을', '위', '한', '액면가', '스톡옵션', '의',...","['간단', '소개', '혜움랩스', '는', '세무', '재무', '경리', '등...","['세무', '알고리즘', '자동화', '설계', '및', '개발', '세무', '..."
1,혜움랩스,"['프론트엔드', '개발자', '주니어']",웹 개발자,"['최소', '2년', '이상', '혹은', '그', '에', '준', '하는', ...","['Vue', '.', 'js', 'React', '.', 'js', 'Angula...",0,https://www.wanted.co.kr/wd/167433,"['#', '몰입', '을', '위', '한', '액면가', '스톡옵션', '의',...","['간단', '소개', '혜움랩스', '는', '세무', '재무', '경리', '등...","['세무', '관련', '서비스', '및', '플랫폼', '의', '프론트엔드', ..."
2,유섭,"['프론트엔드', '개발자', 'Software', 'Engineer', '3년',...",웹 개발자,"['최소', '2년', '이상', '혹은', '그', '에', '준', '하는', ...","['이런', '분', '은', '더욱', '환영', '합니다', '.', '팀', ...",3,https://www.wanted.co.kr/wd/167134,"['자유로운', '휴가', '무제한', '무급', '휴가', '동료', '와', '...","['유섭', '은', '현재', '번역', '서비스', 'Subup', '및', '...","['어떤', '업무', '를', '하게', '되나요', '?', '알트', '에디터..."
3,유섭,"['백엔드', '개발자', 'Backend', 'Software', 'Enginee...",웹 개발자,"['최소', '2년', '이상', '혹은', '그', '에', '준', '하는', ...","['이런', '분', '은', '더욱', '환영', '합니다', '.', '팀', ...",3,https://www.wanted.co.kr/wd/167133,"['자유로운', '휴가', '무제한', '무급', '휴가', '동료', '와', '...","['유섭', '은', '현재', '번역', '서비스', 'Subup', '및', '...","['어떤', '업무', '를', '하게', '되나요', '?', '알트', '에디터..."
4,인투씨엔에스,"['웹', '개발', '인재', 'PHP']",웹 개발자,"['최소', '2년', '이상', '혹은', '그', '에', '준', '하는', ...","['웹', '서비스', '라이브', '런칭', '경험', '자', 'RESTful'...",0,https://www.wanted.co.kr/wd/166922,"['매월', '1회', '패밀리', '데이', '오전', '근무', '세번', '째...","['다음', '과', '같이', '유능하고', '역량', '있는', '인재', '를...","['웹', '기반', '검사', '시스템', '개발', '및', '유지', '보수'..."


In [6]:
r['자격요건'].unique()

array(["['최소', '2년', '이상', '혹은', '그', '에', '준', '하는', '해당', '업무', '경험', 'Python', '필수', '웹', '프레임워크', 'Django', '에', '능숙하신', '분', '테스트', '코드', '작성', '의', '필요성', '을', '이해', '하고', '실천', '하시는', '분', '코드', '리뷰', '와', '페어', '프로그래밍', '등', '팀', '내', '협업', '의', '필요성', '을', '이해', '하고', '실천', '하시는', '분', '개발', '기획', '세무', '운영', 'CS', '등', '다양한', '직', '군', '의', '동료', '들', '과', '원활한', '소통', '이', '가능하신', '분', 'Tech', 'Specs', 'Language', 'Python', '3', 'Framework', 'Django', 'rest', 'framework', 'Flask', 'FastAPI', 'Celery', 'Data', 'storage', 'PostgreSQL', 'Redis', 'DynamoDB', 'S', '3', 'Deployment', 'Github', 'action', 'Docker', 'ECS', 'EKS', 'Cloud', 'platform', 'AWS', 'VCS', 'Github', 'Communication', 'Slack', 'Documentation', 'Notion', 'G', 'Suite', 'Jira', 'Issue', 'Tracker', 'JIRA', 'Github', 'JavaScript', 'TypeScript', 'React', '.', 'js', 'Vue', '.', 'JS']"],
      dtype=object)

In [7]:
r = pd.read_csv('../Data_Pretreatment/Data_result/pre_result.csv')

r.head(5)

,회사명,공고명,직무내용,자격요건,우대조건,경력조건,출처 URL,복지,회사소개,주요업무
0,혜움랩스,Python Developer (백엔드 주니어),웹 개발자,"• 최소 2년 이상 혹은 그에 준하는 해당 업무 경험\r\n• Python(필수),...","• MSA 기반의 시스템 개발 경험이 있으신 분\r\n• AWS 를 활용한 개발, ...",2,https://www.wanted.co.kr/wd/167436,# 몰입을 위한\r\n• 액면가 스톡옵션의 기회가 열려있습니다. (2023년 7월 ...,"- 간단소개\r\n혜움랩스는 세무,재무,경리 등 AI기반 비서형 금융 서비스를 Fa...",• 세무 알고리즘 자동화 설계 및 개발\r\n• 세무팀과 커뮤니케이션\r\n• AW...
1,혜움랩스,프론트엔드 개발자 (주니어),웹 개발자,"• 최소 2년 이상 혹은 그에 준하는 해당 업무 경험\r\n• Python(필수),...","• Vue.js, React.js, Angular 등 SPA 개발 경험이 있는 분\...",0,https://www.wanted.co.kr/wd/167433,# 몰입을 위한\r\n• 액면가 스톡옵션의 기회가 열려있습니다. (2023년 7월 ...,"- 간단소개\r\n혜움랩스는 세무,재무,경리 등 AI기반 비서형 금융 서비스를 Fa...",• 세무 관련 서비스 및 플랫폼의 프론트엔드 에플리케이션 개발\r\n• 요구사항 분...
2,유섭,프론트엔드 개발자 Software Engineer (3년 이상) 사본,웹 개발자,"• 최소 2년 이상 혹은 그에 준하는 해당 업무 경험\r\n• Python(필수),...",[이런 분은 더욱 환영합니다.]\r\n• 팀 리드 경험이 있으신 분\r\n• TTS...,3,https://www.wanted.co.kr/wd/167134,• 자유로운 휴가 (무제한 무급 휴가) \r\n• 동료와 원활한 협업을 위해 11시...,유섭은 현재 번역 서비스 (Subup) 및 자막 편집 에디터(Alt-Editor)를...,[어떤 업무를 하게 되나요?] \r\n• 알트에디터 (영상 번역 | 자막생성 에디...
3,유섭,백엔드 개발자 (Backend Software Engineer | 3년 이상) 사본,웹 개발자,"• 최소 2년 이상 혹은 그에 준하는 해당 업무 경험\r\n• Python(필수),...",[이런 분은 더욱 환영합니다.]\r\n• 팀 리드 경험이 있으신 분\r\n• TTS...,3,https://www.wanted.co.kr/wd/167133,• 자유로운 휴가 (무제한 무급 휴가) \r\n• 동료와 원활한 협업을 위해 11시...,유섭은 현재 번역 서비스 (Subup) 및 자막 편집 에디터(Alt-Editor)를...,[어떤 업무를 하게 되나요?] \r\n• 알트에디터 (영상 번역 | 자막생성 에디...
4,인투씨엔에스,웹개발 인재(PHP),웹 개발자,"• 최소 2년 이상 혹은 그에 준하는 해당 업무 경험\r\n• Python(필수),...",웹 서비스 라이브 런칭 경험자\r\nRESTful API 설계 및 개발경험\r\nG...,0,https://www.wanted.co.kr/wd/166922,매월 1회 패밀리데이(오전근무/세번째 금요일)\r\n패밀리데이 상품권 지급(연 60...,• 다음과 같이 유능하고 역량있는 인재를 채용합니다.\r\n당사와 미래를 함께 이끌...,• 웹기반 검사시스템 개발 및 유지보수\r\n\r\n• 신규 프로젝트 개발


In [9]:
r = pd.read_csv('../Data_Pretreatment/Data_result/pre_jumpit.csv')

r.head(5)


,회사명,지원기간,공고명,직무내용,자격요건,우대조건,경력조건,기술 스택,출처 URL,복지,회사소개,주요업무
0,"미디어로그는 MVNO(U+유모바일)사업, 중고폰사업(셀로), PP(방송채널)사업과 ...",상시,미디어로그(LG그룹) 웹 하드 3.0 SM 경력 채용,서버/백엔드 개발자,"• Java, Spring, Spring Boot, JSP 개발 스킬 필수\n• O...","• LG U+ SI/SM 업무 경험\n• 서버 간 연동 처리 경험 (restful,...",10,Java\nJSP\nSpring Boot\nMariaDB\nOracle\nLinux,https://www.jumpit.co.kr/position/16607,• 통신비 지원\n- U+유모바일 음성무제한 요금제 지원\n\n• 자녀 학비 지원\...,"미디어로그는 MVNO(U+유모바일)사업, 중고폰사업(셀로), PP(방송채널)사업과 ...",• 웹 하드 3.0 운영 유지보수 기능 개발\n• VOC 대응\n• 장애 대응 및 ...
1,대한민국 No.1\n지바이크는 전국 50개 도시에서 전동킥보드 공유서비스 ‘지쿠터’...,상시,[지쿠터] PHP / Laravel,서버/백엔드 개발자,• 해당 경력 3년 이상\n• Laravel 사용 경험이 있으신 분\n• 코드리뷰에...,• 대규모 트래픽 서비스 환경에서 개발 및 운영 경험이 있으신 분\n• 대용량 배치...,3,Laravel\nApache HTTP Server\nREST API\nPHP\nRD...,https://www.jumpit.co.kr/position/1583,• 영어이름으로 실현하는 수평적 조직문화!\n 지쿠터엔 직위 직급 없이 영문 닉네임...,대한민국 No.1\n지바이크는 전국 50개 도시에서 전동킥보드 공유서비스 ‘지쿠터’...,• 지바이크 내 서비스 서버 개발\n• 지바이크 내 운영 시스템 개발
2,핏토(Fitor)는 장소와 시간에 구애받지 않고 내가 원하는 트레이너를 찾아 운동을...,상시,"백엔드, 서버 개발자(경력 3년 이상)",서버/백엔드 개발자,• Backend 개발 경력 3년 이상 혹은 그에 준하는 실력을 보유한 분\n• J...,• Web RTC등의 스트리밍 서비스 개발 경험이 있는 분\n• 네트워크/앱 보안에...,3,JavaScript\nSpring Data JPA\nJava,https://www.jumpit.co.kr/position/11720,"• 원거리 출퇴근자 재택 근무 및 선택적 탄력근무제\n• 4대 보험가입, 근로기준법...",핏토(Fitor)는 장소와 시간에 구애받지 않고 내가 원하는 트레이너를 찾아 운동을...,"• 실시간 매칭, 1:1 화상, 메시징, 예약 시스템 등 모든 API 서비스 개발\..."
3,포스트보이저(POST VOYAGER)는 지난 13년 간 아바타 소셜 앱 서비스 분야...,상시,Java/ Kotlin 개발자,서버/백엔드 개발자,• Java 또는 Kotlin 개발 경력 5년 이상\n• Linux에서 웹 서비스 ...,"• MSA 개발/운영 경험\n• Go, Rust, Koltin 개발 경험\n• 대용...",5,Java\nLinux\nKotlin\nREST API\nAWS\nGCP\nGit\n...,https://www.jumpit.co.kr/position/13204,• 산전/산후휴가 / 육아휴가\n• 유급휴가 / 경조휴가 / 기념일 반차\n• 법정...,포스트보이저(POST VOYAGER)는 지난 13년 간 아바타 소셜 앱 서비스 분야...,• 자사 블록체인 기반 서비스 개발\n• 클라이언트와의 통신을 위한 REST API...
4,"[회사소개]\n넛지헬스케어는 행동경제학 이론을 실생활에 적용하여 비만, 당뇨, 고혈...",상시,[캐시워크] 백엔드/서버 개발자,서버/백엔드 개발자,• 백엔드 개발 경력 3년 이상이신 분\n• Node.js로 상용 서비스 배포해 본...,• 모바일 B2C 서비스 개발 경력이 있는 분\n• Typescript 사용 경험이...,3,Node.js\nNoSql\nCachet\nAWS\nREST API\nGit\nTy...,https://www.jumpit.co.kr/position/11982,• 프리미엄 프라이데이 운영 (매월 마지막 주 금요일 오전 근무만)\n• 죠샌드위치...,"[회사소개]\n넛지헬스케어는 행동경제학 이론을 실생활에 적용하여 비만, 당뇨, 고혈...",• 앱 다운로드 1000만 이상 / 동시 접속자 100만의 대규모 서비스의 서버 운...
